<a href="https://colab.research.google.com/github/Sujal-Patnaik/SVM-Classifier-from-Scratch/blob/main/SVM(Classifier).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

class SVM:
    def __init__(self, kernel='linear', C=1.0, tol=1e-3, max_passes=5, max_iter=1000, degree=3, gamma=None):
        self.C = C
        self.tol = tol
        self.max_passes = max_passes
        self.max_iter = max_iter
        self.kernel_type = kernel
        self.degree = degree
        self.gamma = gamma
        self.b = 0.0

    def kernel(self, x, y):
        if self.kernel_type == 'linear':
            return x @ y.T
        elif self.kernel_type == 'polynomial':
            return (x @ y.T + 1) ** self.degree
        elif self.kernel_type == 'rbf':
            if self.gamma is None:
                self.gamma = 1 / x.shape[1]
            x_norm = np.sum(x**2, axis=-1).reshape(-1, 1)
            y_norm = np.sum(y**2, axis=-1).reshape(1, -1)
            return np.exp(-self.gamma * (x_norm + y_norm - 2 * x @ y.T))
        else:
            raise ValueError("Unknown kernel")

    def fit(self, X, y):
        n_samples, n_features = X.shape
        y = y.astype(np.float64)
        y[y == 0] = -1  # convert labels to {-1, 1}

        self.X = X
        self.y = y
        self.alphas = np.zeros(n_samples)  #initialize the alphas as a 0 vector
        self.b = 0.0 #initialize b to 0
        self.K = self.kernel(X, X)

        # Initialize the error cache
        self.E_cache = self.project(self.X) - self.y

        passes = 0
        iters = 0

        while passes < self.max_passes and iters < self.max_iter:
            num_changed_alphas = 0
            for i in range(n_samples):
                # E_i from the error Cache
                E_i = self.E_cache[i]

                #KKT condition checking
                cond1 = (y[i] * E_i < -self.tol) and (self.alphas[i] < self.C)
                cond2 = (y[i] * E_i > self.tol) and (self.alphas[i] > 0)

                #The i is chosen for which alpha[i] violates KKT(Karn Kush Tucker Conditions)
                if cond1 or cond2:
                    non_zero_alphas = np.where((self.alphas > 0) & (self.alphas < self.C))[0]
                    if len(non_zero_alphas) > 1:
                        # E_list contains the errors of all the non-zero alphas
                        E_list = self.E_cache[non_zero_alphas]
                        #j is chosen to maximize |E_i - E_j| if possible (i.e., from non-bound alphas).
                        j = non_zero_alphas[np.argmax(np.abs(E_list - E_i))]
                    else:
                        #Otherwise, it is randomly selected.
                        j = np.random.randint(0, n_samples)
                        while j == i:
                            j = np.random.randint(0, n_samples)

                    # Extracting E_j from the Error Cache
                    E_j = self.E_cache[j]

                    alpha_i_old, alpha_j_old = self.alphas[i], self.alphas[j]

                    #Finding the Lower and Upper Bounds to clip the alpha[j]
                    if y[i] != y[j]:
                        L = max(0, alpha_j_old - alpha_i_old)
                        H = min(self.C, self.C + alpha_j_old - alpha_i_old)
                    else:
                        L = max(0, alpha_i_old + alpha_j_old - self.C)
                        H = min(self.C, alpha_i_old + alpha_j_old)

                    if L == H:
                        continue

                    eta = 2.0 * self.K[i, j] - self.K[i, i] - self.K[j, j]
                    if eta >= 0:
                        continue  #This ensures that we update the pair (i,j) only if the curvature of the objective function is suitable for descent

                    self.alphas[j] -= y[j] * (E_i - E_j) / eta #updating the alpha[j]
                    self.alphas[j] = np.clip(self.alphas[j], L, H)

                    if abs(self.alphas[j] - alpha_j_old) < 1e-5:
                        continue

                    self.alphas[i] += y[i] * y[j] * (alpha_j_old - self.alphas[j]) #updating the alpha[i] after alpha[j] is found out

                    b1 = self.b - E_i - y[i] * (self.alphas[i] - alpha_i_old) * self.K[i, i] - y[j] * (self.alphas[j] - alpha_j_old) * self.K[i, j]
                    b2 = self.b - E_j - y[i] * (self.alphas[i] - alpha_i_old) * self.K[i, j] - y[j] * (self.alphas[j] - alpha_j_old) * self.K[j, j]

                    if 0 < self.alphas[i] < self.C:
                        self.b = b1    #This means alpha[i] is a support vector
                    elif 0 < self.alphas[j] < self.C:
                        self.b = b2    #This means alpha[j] is a support vector
                    else:
                        self.b = (b1 + b2) / 2  #Since neither alpha[i] nor alpha[j] are support vectors, b1 and b2 are averaged to get b

                    # Updating the Error Cache
                    delta_b = self.b - self.b_old if hasattr(self, 'b_old') else self.b
                    self.E_cache += (y[i] * (self.alphas[i] - alpha_i_old) * self.K[:, i] +
                                     y[j] * (self.alphas[j] - alpha_j_old) * self.K[:, j] + delta_b)
                    self.b_old = self.b

                    num_changed_alphas += 1

            if num_changed_alphas == 0:
                passes += 1
            else:
                passes = 0

            iters += 1

        self.support_ = np.where(self.alphas > 1e-5)[0]
        self.alpha_sv = self.alphas[self.support_]
        self.X_sv = X[self.support_]
        self.y_sv = y[self.support_]
        print(f"Training complete (SMO). Support vectors found: {len(self.support_)}")

    def _decision_function_index(self, i):
        return np.sum(self.alphas * self.y * self.K[i]) + self.b

    def _decision_function(self, X):
        return np.dot((self.alphas * self.y), self.kernel(X, self.X).T) + self.b

    def project(self, X):
        return self._decision_function(X)

    def predict(self, X):
        return np.sign(self.project(X))

    def fit_gd(self, X, y, lr=0.001, epochs=1000):
        n_samples, n_features = X.shape
        y = y.astype(np.float64)
        y[y == 0] = -1

        self.w = np.zeros(n_features)
        self.b = 0.0

        for epoch in range(epochs):
            margin = y * (X @ self.w + self.b)
            mask = margin < 1

            dw = self.w - self.C * (X[mask].T @ y[mask])
            db = -self.C * np.sum(y[mask])

            self.w -= lr * dw
            self.b -= lr * db

        print("Training complete (vectorized gradient descent).")
        self.support_ = np.where(y * (X @ self.w + self.b) < 1)[0]
        self.X_sv = X[self.support_]
        self.y_sv = y[self.support_]

    def predict_gd(self, X):
        if not hasattr(self, 'w'):
            raise AttributeError("Model has not been trained using gradient descent.")
        return np.sign(X @ self.w + self.b)

    def project_gd(self, X):
        return X @ self.w + self.b

In [2]:
from sklearn.datasets import load_digits
digits = load_digits()
mask = (digits.target == 1) | (digits.target == 7)
X = digits.data[mask, :10]  # First 10 features
y = digits.target[mask]
y = np.where(y == 1, 1, -1)  # 1 and -1


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# scratch built SVM
custom_svm = SVM(kernel='linear', C=1.0)
custom_svm.fit(X_train, y_train)
y_pred_custom = custom_svm.predict(X_test)

# Scikit-learn's SVM
from sklearn.svm import SVC
sklearn_svm = SVC(kernel='linear', C=1.0)
sklearn_svm.fit(X_train, y_train)
y_pred_sklearn = sklearn_svm.predict(X_test)


Training complete (SMO). Support vectors found: 63


In [4]:
from sklearn.metrics import accuracy_score, classification_report

print("Scratch SVM Accuracy:", accuracy_score(y_test, y_pred_custom))
print("Sklearn SVM Accuracy:", accuracy_score(y_test, y_pred_sklearn))

print("\nScratch SVM Report:\n", classification_report(y_test, y_pred_custom))
print("\nSklearn SVM Report:\n", classification_report(y_test, y_pred_sklearn))


Scratch SVM Accuracy: 0.8623853211009175
Sklearn SVM Accuracy: 0.8715596330275229

Scratch SVM Report:
               precision    recall  f1-score   support

          -1       0.83      0.88      0.85        50
           1       0.89      0.85      0.87        59

    accuracy                           0.86       109
   macro avg       0.86      0.86      0.86       109
weighted avg       0.86      0.86      0.86       109


Sklearn SVM Report:
               precision    recall  f1-score   support

          -1       0.85      0.88      0.86        50
           1       0.89      0.86      0.88        59

    accuracy                           0.87       109
   macro avg       0.87      0.87      0.87       109
weighted avg       0.87      0.87      0.87       109



In [5]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


X, y = load_breast_cancer(return_X_y=True)

y = np.where(y == 0, -1, 1)


scaler = StandardScaler()
X = scaler.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

#Training the sklearn SVM
clf = SVC(C=1.0, kernel='linear')
clf.fit(X_train, y_train)
y_pred_sklearn = clf.predict(X_test)

acc_sklearn = accuracy_score(y_test, y_pred_sklearn)
print(f"Sklearn SVM Accuracy: {acc_sklearn:.4f}")

#Training the SVM built from scratch
my_svm = SVM(C=1.0)
my_svm.fit(X_train, y_train)
y_pred_custom = my_svm.predict(X_test)

acc_custom = accuracy_score(y_test, y_pred_custom)
print(f"Scratch SVM Accuracy: {acc_custom:.4f}")


Sklearn SVM Accuracy: 0.9561
Training complete (SMO). Support vectors found: 36
Scratch SVM Accuracy: 0.9561


In the below 2 cells I have used One Vs Rest approach for multiclass classification using my SVM from scratch. In this approach we use N binary classifiers for N number of classes and combine their results.

In [6]:
from sklearn.datasets import load_iris
data = load_iris()
X = data.data
y = data.target  # classes: 0, 1, 2


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


classes = np.unique(y_train)
custom_svms = []

# Training one binary SVM per class
for cls in classes:
    y_binary = np.where(y_train == cls, 1, -1)
    clf = SVM(kernel='rbf', C=1.0)
    clf.fit(X_train, y_binary)
    custom_svms.append(clf)

# Predict using decision function from all classifiers
custom_preds = []
for x in X_test:
    scores = [clf.project(x.reshape(1, -1))[0] for clf in custom_svms]
    pred = np.argmax(scores)
    custom_preds.append(pred)

custom_acc = accuracy_score(y_test, custom_preds)
print(f"\nScratch SVM (OvR) Accuracy: {custom_acc:.4f}")

# ========== Sklearn SVC ==========
sklearn_clf = SVC(kernel='rbf')
sklearn_clf.fit(X_train, y_train)
sklearn_preds = sklearn_clf.predict(X_test)
sklearn_acc = accuracy_score(y_test, sklearn_preds)
print(f"Sklearn SVC Accuracy: {sklearn_acc:.4f}")



Training complete (SMO). Support vectors found: 12
Training complete (SMO). Support vectors found: 32
Training complete (SMO). Support vectors found: 31

Scratch SVM (OvR) Accuracy: 1.0000
Sklearn SVC Accuracy: 1.0000


In [7]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import numpy as np



# Loading the digits dataset
digits = load_digits()
X = digits.data
y = digits.target


# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train custom SVMs with One-vs-Rest (OvR)
classes = np.unique(y_train)
custom_svms = []

for cls in classes:
    # Create binary labels: 1 for current class, -1 for others
    y_binary = np.where(y_train == cls, 1, -1)

    # Train a custom SVM for each class
    clf = SVM(kernel='rbf', C=10.0, gamma=0.001)
    clf.fit(X_train, y_binary)
    custom_svms.append(clf)

# Make predictions using the OvR(One Vs Rest) strategy
custom_preds = []
for x in X_test:
    # Get scores from all 10 classifiers
    scores = [clf.project(x.reshape(1, -1))[0] for clf in custom_svms]

    # Predict the class with the highest score
    pred = np.argmax(scores)
    custom_preds.append(pred)

# Checking the scratch SVM's accuracy
custom_acc = accuracy_score(y_test, custom_preds)
print(f"\nScratch SVM (OvR with SMO) Accuracy on Digits: {custom_acc:.4f}")

## skicit learn SVM accuracy
# Using the same hyperparameters for a fair comparison
sklearn_clf = SVC(kernel='rbf', C=10.0, gamma=0.001)
sklearn_clf.fit(X_train, y_train)
sklearn_preds = sklearn_clf.predict(X_test)
sklearn_acc = accuracy_score(y_test, sklearn_preds)
print(f"Sklearn SVC Accuracy on Digits: {sklearn_acc:.4f}")

Training complete (SMO). Support vectors found: 81
Training complete (SMO). Support vectors found: 149
Training complete (SMO). Support vectors found: 169
Training complete (SMO). Support vectors found: 156
Training complete (SMO). Support vectors found: 146
Training complete (SMO). Support vectors found: 146
Training complete (SMO). Support vectors found: 111
Training complete (SMO). Support vectors found: 141
Training complete (SMO). Support vectors found: 226
Training complete (SMO). Support vectors found: 181

Scratch SVM (OvR with SMO) Accuracy on Digits: 0.9889
Sklearn SVC Accuracy on Digits: 0.9907
